In [6]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
import numpy as np
from enum import Enum
from Gan import *
from GeneralArc import *
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import init
import datetime
import os
from torch.utils.tensorboard import SummaryWriter
import torchvision

from Loss import *
import sys 
sys.path.append("..") 
from data_loader import data_loader
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [4]:
class ARGS:
    def __init__(self):
        self.datapath="F:\dataset\multi_focus"
        self.init_size=32
        self.final_size=128
        self.device=th.device("cuda:0" if th.cuda.is_available() else "cpu")
        self.adam_beta1=0.9
        self.adam_beta2=0.999
        self.g_lr=0.0001
        self.d_lr=0.0004
        self.generator_optim_file=None
        self.discriminator_optim_file=None
        self.loss_function="RelativeplusContentLoss"
        self.alpha=3
        self.beta=2
        self.num_epochs=1 #
        self.checkpoint_factor=1#
        self.feedback_factor=5#
        self.data_percentage=100
        self.num_samples=2#
        self.sample_dir="F:/dataset/multi_focus/MSG-MFGAN/recordings/images"
        self.batch_size=1
        self.start=1
        self.loss_dir="F:/dataset/multi_focus/MSG-MFGAN/recordings/loss"
        self.model_dir="F:/dataset/multi_focus/MSG-MFGAN/recordings/model"
        
args=ARGS()        
datapath=args.datapath
init_size=args.init_size
final_size=args.final_size
device=args.device

#get the data
data=data_loader.MultiFocusDataset(datapath,mode="train",size=final_size)
data=DataLoader(data, batch_size=args.batch_size, shuffle=True)
testdata=data_loader.MultiFocusDataset(datapath,mode="test",size=final_size)
testdata=DataLoader(testdata, batch_size=args.num_samples, shuffle=True)
#get the MSG_MFGAN model
msg_mfgan=MSG_MFGAN(init_size=init_size,final_size=final_size,device=device)

gen_optim = th.optim.Adam(msg_mfgan.gen.parameters(), args.g_lr,
                              [args.adam_beta1, args.adam_beta2])
dis_optim = th.optim.Adam(msg_mfgan.dis.parameters(), args.d_lr,
                              [args.adam_beta1, args.adam_beta2])

if args.generator_optim_file is not None:
        gen_optim.load_state_dict(th.load(args.generator_optim_file))

if args.discriminator_optim_file is not None:
        dis_optim.load_state_dict(th.load(args.discriminator_optim_file))
        
loss_name = args.loss_function.lower()
if loss_name=="relativepluscontentloss":
    loss=RelativeplusContentLoss(device=args.device,dis=msg_mfgan.dis,alpha=args.alpha,beta=args.beta)

msg_mfgan.train(
        data,
        gen_optim,
        dis_optim,
        loss_fn=loss,
        testdata=testdata,
        num_epochs=args.num_epochs,
        checkpoint_factor=args.checkpoint_factor,
        data_percentage=args.data_percentage,
        feedback_factor=args.feedback_factor,
        num_samples=args.num_samples,
        sample_dir=args.sample_dir,
        save_dir=args.model_dir,
        loss_dir=args.loss_dir,
        start=args.start
    )
# train(self, data, gen_optim, dis_optim, loss_fn,testdata,
#               start=1, num_epochs=12, feedback_factor=10, checkpoint_factor=1,
#               data_percentage=100, num_samples=4,
#               loss_dir='muti_recordings/loss', sample_dir='muti_recordings/images',
#               save_dir="./models"):

        

Starting the training process ... 

Epoch: 1


KeyboardInterrupt: 